# 😄 表情包生成器 - Qwen Image Edit

基于 Qwen-Image-Edit 模型的智能表情包生成工具

In [ ]:
# 安装必要的依赖
!pip install gradio diffusers modelscope accelerate transformers  openai

In [ ]:
import gradio as gr
import torch
from PIL import Image
import os
from diffusers import QwenImageEditPipeline
from modelscope import snapshot_download
import numpy as np
import datetime
import glob
from typing import List, Optional
import sys
from pathlib import Path

# 添加工具模块路径
sys.path.append('./src/examples/tools')

# 从 .env 文件加载环境变量
def load_env_file():
    env_file = Path('.env')
    if env_file.exists():
        with open(env_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip().strip('"').strip("'")
                    os.environ[key] = value
        print("✅ 已从 .env 文件加载环境变量")
        return True
    return False

# 加载环境变量
env_loaded = load_env_file()

# 检查API密钥是否设置
if 'OPENAI_API_KEY' not in os.environ:
    print("⚠️ 未找到 OPENAI_API_KEY")
    if not env_loaded:
        print("💡 请在项目根目录创建 .env 文件，内容如下:")
        print("   OPENAI_API_KEY=your-api-key-here")
    else:
        print("💡 请在 .env 文件中添加 OPENAI_API_KEY=your-api-key-here")
    print("   或者设置系统环境变量 OPENAI_API_KEY")
else:
    print("✅ OPENAI_API_KEY 已设置")

# 导入专用提示词生成工具
try:
    from ip_creation_prompt_utils import (
        generate_creative_prompt, 
        get_subject_suggestions, 
        get_action_suggestions,
        validate_inputs
    )
    print("✅ 成功导入专用提示词生成工具")
    IP_CREATION_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ 专用提示词生成工具导入失败: {e}")
    print("💡 请确保已设置 OPENAI_API_KEY 环境变量")
    IP_CREATION_AVAILABLE = False

In [ ]:
# 设置设备和数据类型
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.bfloat16
    print(f"✅ 使用 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = "cpu"
    torch_dtype = torch.float32
    print("⚠️ 使用 CPU（建议使用GPU以获得更好性能）")

print(f"🔧 设备: {device}, 数据类型: {torch_dtype}")

# 创建输出目录
output_dir = "./emoji_outputs"
os.makedirs(output_dir, exist_ok=True)
print(f"📁 输出目录: {output_dir}")

In [ ]:
# 下载并加载 Qwen-Image-Edit 模型
model_id = "Qwen/Qwen-Image-Edit"
local_dir = './models/Qwen-Image-Edit'

# 检查模型是否已存在
if not os.path.exists(local_dir):
    print(f"📥 开始下载 {model_id} 模型...")
    os.makedirs(os.path.dirname(local_dir), exist_ok=True)
    snapshot_download(model_id, local_dir=local_dir)
    print(f"✅ 模型下载完成: {local_dir}")
else:
    print(f"✅ 模型已存在: {local_dir}")

# 加载管道
print("🔄 正在加载图像编辑管道...")
pipeline = QwenImageEditPipeline.from_pretrained(
    local_dir, 
    torch_dtype=torch_dtype,
    use_safetensors=True,
    device_map="balanced"
)
print("✅ 模型加载完成")

In [ ]:
def generate_emoji(image, prompt, seed=42, steps=50, cfg_scale=4.0):
    """
    生成表情包
    """
    try:
        # 处理输入图像
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        # 编辑参数
        inputs = {
            "image": image,
            "prompt": prompt,
            "generator": torch.manual_seed(seed),
            "true_cfg_scale": cfg_scale,
            "negative_prompt": "blurry, low quality, distorted",
            "num_inference_steps": steps,
        }
        
        # 执行编辑
        with torch.inference_mode():
            output = pipeline(**inputs)
            edited_image = output.images[0]
        
        # 保存生成的图像
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"emoji_{timestamp}_{seed}.png"
        filepath = os.path.join(output_dir, filename)
        edited_image.save(filepath)
        
        return edited_image
        
    except Exception as e:
        print(f"❌ 生成表情包时出错: {str(e)}")
        return image

def get_history_images():
    """
    获取历史生成的图片
    """
    image_files = glob.glob(os.path.join(output_dir, "*.png"))
    image_files.sort(key=os.path.getmtime, reverse=True)  # 按修改时间倒序
    return image_files[:12]  # 返回最新的12张图片

def load_history_image(image_path):
    """
    加载历史图片
    """
    if image_path and os.path.exists(image_path):
        return Image.open(image_path)
    return None

def generate_ip_creation_prompt(subject, simple_prompt):
    """
    生成专用的IP创作提示词
    """
    if not IP_CREATION_AVAILABLE:
        return f"Make the {subject} {simple_prompt}"
    
    if not subject or not subject.strip():
        return "请先输入图片主体"
    
    if not simple_prompt or not simple_prompt.strip():
        return "请先输入简单动作"
    
    # 验证输入
    is_valid, error_msg = validate_inputs(subject.strip(), simple_prompt.strip())
    if not is_valid:
        return error_msg
    
    try:
        generated = generate_creative_prompt(subject.strip(), simple_prompt.strip())
        return generated
    except Exception as e:
        print(f"⚠️ 专用提示词生成失败: {e}")
        return f"Make the {subject} {simple_prompt}"

In [ ]:
# 创建表情包生成器界面
def create_emoji_generator():
    with gr.Blocks(title="表情包生成器", theme=gr.themes.Soft() ) as demo:
        gr.Markdown(
            """
            # 😄 表情包生成器
            
            使用 Qwen-Image-Edit 模型，将普通图片转换为有趣的表情包！
            
            **使用步骤：** 上传图片 → 输入图片主体 → 输入简单动作 → 生成专用提示词 → 生成表情包
            
            **新功能：** 专用提示词生成器 - 只需输入主体和简单动作，AI会自动生成详细的创意提示词！
            """
        )
        
        with gr.Row():
            # 左侧：输入区域
            with gr.Column(scale=1):
                # 图片输入
                input_image = gr.Image(
                    label="图片输入",
                    type="pil",
                    height=300
                )
                
                # 主体询问区域
                gr.Markdown("**图片主体**")
                subject_input = gr.Textbox(
                    label="",
                    placeholder="请输入图片中的主体，例如：小熊、猫、人物等...",
                    lines=1
                )
                
                # 主体建议按钮
                with gr.Row():
                    btn_bear = gr.Button("小熊", size="sm")
                    btn_cat = gr.Button("猫", size="sm")
                    btn_dog = gr.Button("狗", size="sm")
                    btn_rabbit = gr.Button("兔子", size="sm")
                    btn_person = gr.Button("人物", size="sm")
                
                # 提示词区域
                gr.Markdown("**简单动作**")
                
                # 快速动作按钮
                with gr.Row():
                    btn_paint = gr.Button("画画", size="sm")
                    btn_guitar = gr.Button("弹吉他", size="sm")
                    btn_astronaut = gr.Button("宇航员", size="sm")
                    btn_magician = gr.Button("魔法师", size="sm")
                
                with gr.Row():
                    btn_read = gr.Button("读书", size="sm")
                    btn_cook = gr.Button("做饭", size="sm")
                    btn_sleep = gr.Button("睡觉", size="sm")
                    btn_dance = gr.Button("跳舞", size="sm")
                
                # 简单动作输入
                simple_prompt_input = gr.Textbox(
                    label="",
                    placeholder="输入简单动作，例如：画画、弹吉他、宇航员...",
                    lines=1
                )
                
                # 专用提示词生成按钮
                with gr.Row():
                    generate_prompt_button = gr.Button(
                        "🎯 生成专用提示词", 
                        variant="secondary", 
                        size="sm",
                        visible=IP_CREATION_AVAILABLE
                    )
                    if not IP_CREATION_AVAILABLE:
                        gr.Markdown("💡 *设置 OPENAI_API_KEY 环境变量以启用专用提示词生成功能*", elem_id="ip-creation-tip")
                
                # 生成的详细提示词显示
                prompt_input = gr.Textbox(
                    label="生成的详细提示词",
                    placeholder="这里将显示生成的详细英文提示词...",
                    lines=4,
                    interactive=True
                )
                
                
                # 高级参数
                gr.Markdown("**高级参数**")
                with gr.Accordion("参数设置", open=False):
                    seed_input = gr.Slider(0, 1000, value=42, step=1, label="随机种子")
                    steps_input = gr.Slider(10, 50, value=30, step=5, label="推理步数")
                    cfg_scale_input = gr.Slider(1.0, 8.0, value=4.0, step=0.5, label="CFG缩放")
                
                generate_button = gr.Button("🎨 生成表情包", variant="primary", size="lg")
            
            # 右侧：输出区域
            with gr.Column(scale=1):
                # 图片输出
                output_image = gr.Image(label="图片输出", height=300)
                
                # 历史图片 - 占满剩余空间
                gr.Markdown("**历史图片**")
                history_gallery = gr.Gallery(
                        label="",
                        show_label=False,
                        elem_id="gallery",
                        columns=3,
                        rows=4,
                        height=500,
                        object_fit="contain",
                        container=True,
                        allow_preview=True
                    )
        
        # 处理函数
        def process_generation(image, prompt, seed, steps, cfg_scale, progress=gr.Progress(track_tqdm=True)):
            if image is None:
                gr.Warning("请先上传图片！")
                return None, get_history_images()
            if not prompt.strip():
                gr.Warning("请输入提示词！")
                return None, get_history_images()
            
            try:
                # 生成表情包
                result_image = generate_emoji(image, prompt, seed, steps, cfg_scale)
                # 更新历史图片
                history_images = get_history_images()
                return result_image, history_images
                
            except Exception as e:
                gr.Error(f"生成失败: {str(e)}")
                return None, get_history_images()
        
        def update_history():
            return get_history_images()
        
        def process_generate_ip_prompt(subject, simple_prompt):
            """
            处理专用提示词生成
            """
            if not subject or not subject.strip():
                gr.Warning("请先输入图片主体！")
                return ""
            
            if not simple_prompt or not simple_prompt.strip():
                gr.Warning("请先输入简单动作！")
                return ""
            
            try:
                generated_prompt = generate_ip_creation_prompt(subject, simple_prompt)
                gr.Info("✅ 专用提示词生成完成！")
                return generated_prompt
            except Exception as e:
                gr.Error(f"专用提示词生成失败: {str(e)}")
                return f"Make the {subject} {simple_prompt}"
        
        # 绑定事件
        generate_button.click(
            fn=process_generation,
            inputs=[input_image, prompt_input, seed_input, steps_input, cfg_scale_input],
            outputs=[output_image, history_gallery]
        )
        
        # 主体建议按钮绑定
        subject_suggestions = {
            btn_bear: "小熊",
            btn_cat: "猫",
            btn_dog: "狗",
            btn_rabbit: "兔子",
            btn_person: "人物"
        }
        
        for btn, subject in subject_suggestions.items():
            btn.click(fn=lambda x=subject: x, outputs=[subject_input])
        
        # 快速动作按钮绑定
        action_suggestions = {
            btn_paint: "画画",
            btn_guitar: "弹吉他",
            btn_astronaut: "宇航员",
            btn_magician: "魔法师",
            btn_read: "读书",
            btn_cook: "做饭",
            btn_sleep: "睡觉",
            btn_dance: "跳舞"
        }
        
        for btn, action in action_suggestions.items():
            btn.click(fn=lambda x=action: x, outputs=[simple_prompt_input])
        
        # 专用提示词生成按钮绑定
        if IP_CREATION_AVAILABLE:
            generate_prompt_button.click(
                fn=process_generate_ip_prompt,
                inputs=[subject_input, simple_prompt_input],
                outputs=[prompt_input]
            )
        
        
        # 页面加载时更新历史图片
        demo.load(fn=update_history, outputs=[history_gallery])
    
    return demo

In [ ]:
print("🚀 启动表情包生成器...")
    
demo = create_emoji_generator()

# 启动 Gradio 应用
demo.launch(
    share=False,          # 不创建公共链接
    server_name="0.0.0.0",  # 允许外部访问
    server_port=6006,    # 端口号
    show_error=True,     # 显示详细错误信息
    debug=True          # 调试模式
)